In [29]:
## Load The Pickle File
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [37]:
## Load The Trained model,scaler and onehot pickle
model = load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
  label_encoder_gender = pickle.load(file)
with open('scaler.pkl','rb') as file:
  scaler = pickle.load(file)
with open('onehot_encoder.pkl','rb') as file:
  onehot_encoder = pickle.load(file)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [38]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}


In [39]:
geo_encoded = onehot_encoder.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [40]:
input_data_df = pd.DataFrame([input_data])

# Now concatenate the dataframes
input_data = pd.concat([input_data_df.reset_index(drop = True),geo_encoded_df],axis = 1)
input_data = input_data.drop('Geography',axis = 1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [41]:
print(label_encoder_gender.classes_)


['Female' 'Male']


In [42]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [43]:
## Scaling The Input Data
input_data = scaler.transform(input_data)
input_data

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [44]:
## Final Predidction
prediction = model.predict(input_data)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step


array([[0.03274304]], dtype=float32)

In [46]:
predict_proba = prediction[0][0]

In [47]:
if predict_proba > 0.5:
  print('Customer Will Leave The Bank')
else:
  print('Customer Will Not Leave The Bank')

Customer Will Not Leave The Bank
